# Classifier Evaluation Lab

* Copy&paste your model for homework5 model
* Add grid search and train
* Compare performance
* Which one is better? Explain?

## This lab work is in collaboration of AYUSHI BHUJADE and JAINAM MAHESHKUMAR PATEL

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/log_reg/employee-turnover-balanced.csv')
df.head()

,left_company,age,frequency_of_travel,department,commuting_distance,education,satisfaction_with_environment,gender,seniority_level,position,satisfaction_with_job,married_or_single,last_raise_pct,last_performance_rating,total_years_working,years_at_company,years_in_current_job,years_since_last_promotion,years_with_current_supervisor
0,No,37,Travel_Rarely,Sales,16,4,4,Male,2,Sales Executive,3,Divorced,19,3,9,1,0,0,0
1,No,39,Travel_Rarely,Research & Development,3,2,3,Male,2,Laboratory Technician,3,Divorced,15,3,11,10,8,0,7
2,No,52,Travel_Frequently,Research & Development,25,4,3,Female,4,Manufacturing Director,4,Married,22,4,31,9,8,0,0
3,No,50,Non-Travel,Sales,1,3,4,Female,2,Sales Executive,3,Married,12,3,19,18,7,0,13
4,No,44,Travel_Rarely,Research & Development,4,3,4,Male,2,Healthcare Representative,2,Single,12,3,10,5,2,2,3


In [8]:
# insert code here
# here i imported the required items....
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


# here i first define preprocessors for numerical and categorical data
num_pre_processor = Pipeline([
    ('scaler', StandardScaler())
])

cat_pre_processor = Pipeline([
    ('onehot', OneHotEncoder())
])


# here i transfered the column by defining column transformer to preprocessors for specific columns in it....
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pre_processor, numerical_vars),
        ('cat', cat_pre_processor, categorical_vars)
    ])


# here i fitted and transform data using the preprocessing pipeline which i created....
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


# here checking the last thing which is the shapes of the transformed data...
print("Shape of pre_processed TRAINING data:", X_train_preprocessed.shape)
print("Shape of pre_processed TEST data:", X_test_preprocessed.shape)


Shape of pre_processed TRAINING data: (800, 44)
Shape of pre_processed TEST data: (200, 44)


In [9]:
# insert code here
# again here i imported the required items...
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# here i created a pipeline with logistic regression....
log_reg_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])


# fitting the pipeline on the trained data....
log_reg_pipeline.fit(X_train, y_train)


# here i made some predictions for this ....
train_prediction = log_reg_pipeline.predict(X_train)
test_prediction = log_reg_pipeline.predict(X_test)


# here i calculated accuracy with 2 differnet variable each for train and test data...
train_acc = accuracy_score(y_train, train_prediction)
test_acc = accuracy_score(y_test, test_prediction)


# here i print accuracy variables...
print("TRAINING Accuracy:", train_acc)
print("TEST Accuracy:", test_acc)


TRAINING Accuracy: 0.745
TEST Accuracy: 0.715


# grid search 

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV


df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/log_reg/employee-turnover-balanced.csv')


numerical_vars = ['age', 'commuting_distance', 'satisfaction_with_environment', 'last_raise_pct',
                  'last_performance_rating', 'total_years_working', 'years_at_company',
                  'years_in_current_job', 'years_since_last_promotion', 'years_with_current_supervisor']

categorical_vars = ['frequency_of_travel', 'department', 'education', 'gender', 'seniority_level',
                    'position', 'satisfaction_with_job', 'married_or_single']


X = df.drop(columns=['left_company'])
y = df['left_company']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=124)


num_pre_processor = Pipeline([
    ('scaler', StandardScaler())
])

cat_pre_processor = Pipeline([
    ('onehot', OneHotEncoder())
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pre_processor, numerical_vars),
        ('cat', cat_pre_processor, categorical_vars)
    ])


X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)



log_reg_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=100))
])


log_reg_pipeline.fit(X_train, y_train)


train_prediction = log_reg_pipeline.predict(X_train)
test_prediction = log_reg_pipeline.predict(X_test)
train_probabilities = log_reg_pipeline.predict_proba(X_train)[:, 1]
test_probabilities = log_reg_pipeline.predict_proba(X_test)[:, 1]




# here we defined the hyperparameter grid for grid search and train....
param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10],
    'classifier__penalty': ['l2'],
    'classifier__max_iter': [ 500,1000]
}

# here we tried to create a grid search with cross-validation which is direct...
grid_search = GridSearchCV(log_reg_pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)


# here we fitted the grid search to the training data fro testing....
grid_search.fit(X_train, y_train)



# here we tried to get better estimator and its parameters.....
best_estimator = grid_search.best_estimator_
best_params = grid_search.best_params_


best_estimator.fit(X_train, y_train)

#here we stroed and evaluated the data....
train_prediction = log_reg_pipeline.predict(X_train)
test_prediction = log_reg_pipeline.predict(X_test)
train_probabilities = log_reg_pipeline.predict_proba(X_train)[:, 1]
test_probabilities = log_reg_pipeline.predict_proba(X_test)[:, 1]

train_acc = accuracy_score(y_train, train_prediction)
test_acc = accuracy_score(y_test, test_prediction)
roc_auc_original = roc_auc_score(y_test, test_probabilities)

print("Original Model:")
print("TRAINING Accuracy:", train_acc)
print("TEST Accuracy:", test_acc)
print("ROC AUC:", roc_auc_original)

# Evaluate the best model from grid search
train_prediction_best = best_estimator.predict(X_train)
test_prediction_best = best_estimator.predict(X_test)
train_probabilities_best = best_estimator.predict_proba(X_train)[:, 1]
test_probabilities_best = best_estimator.predict_proba(X_test)[:, 1]

train_acc_best = accuracy_score(y_train, train_prediction_best)
test_acc_best = accuracy_score(y_test, test_prediction_best)
roc_auc_best = roc_auc_score(y_test, test_probabilities_best)

print("\nBest Model (Grid Search):")
print("TRAINING Accuracy:", train_acc_best)
print("TEST Accuracy:", test_acc_best)
print("ROC AUC:", roc_auc_best)


Original Model:
TRAINING Accuracy: 0.745
TEST Accuracy: 0.715
ROC AUC: 0.7616319967842428

Best Model (Grid Search):
TRAINING Accuracy: 0.745
TEST Accuracy: 0.715
ROC AUC: 0.7616319967842428


#### explaination:

- here if we have to determine which model is better, we have to consider various factors and metrics. Accuracy alone is not always the best indicator, especially in imbalanced datasets. There are others options like precision, recall, and F1-score for a more comprehensive evaluation. Additionally, ROC AUC can provide insights into the model's ability to distinguish between classes.

- The model from grid search is likely to perform better because it's fine-tuned to find the best hyperparameters. However, we should still compare the metrics and possibly visualize the results to make a more informed decision. but here we get the same results in both the models.

- here we suggests that the grid search did not lead to an improvement in model performance in terms of accuracy, as it gives the same results for both the models trained and tested, we think Gird search may be used for other options.